# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [48]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
from datasets import load_dataset

# Load the SMS Spam dataset
dataset = load_dataset("sms_spam")

In [50]:
# Perform a train/test split (e.g., 90% train, 10% test)
split_dataset = dataset["train"].train_test_split(test_size=0.1, shuffle=True, seed=23)

splits = ['train', 'test']

# Optional: check sizes
print(f"Sample train data: {split_dataset['train'][0]}")
print(f"Sample test data: {split_dataset['test'][0]}")

Sample train data: {'sms': 'I wanted to ask ü to wait 4 me to finish lect. Cos my lect finishes in an hour anyway.\n', 'label': 0}
Sample test data: {'sms': 'Yup... Hey then one day on fri we can ask miwa and jiayin take leave go karaoke \n', 'label': 0}


In [51]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = split_dataset[split].map(
        lambda x: tokenizer(
            x['sms'],
            truncation=True,
        ), batched=True
    )

print(f"Sample tokenized training data: {tokenized_dataset['train'][0]}")

Sample tokenized training data: {'sms': 'I wanted to ask ü to wait 4 me to finish lect. Cos my lect finishes in an hour anyway.\n', 'label': 0, 'input_ids': [101, 1045, 2359, 2000, 3198, 1057, 2000, 3524, 1018, 2033, 2000, 3926, 3393, 6593, 1012, 2522, 2015, 2026, 3393, 6593, 12321, 1999, 2019, 3178, 4312, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [53]:
for param in model.parameters():
    param.require_grad = True

In [54]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam",
        # Set the learning rate
        learning_rate=2e-5,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",  
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Comment out the train() method since we are only evaluating its performance
# trainer.train()
trainer.evaluate()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6006178259849548,
 'eval_accuracy': 0.8745519713261649,
 'eval_runtime': 1.1221,
 'eval_samples_per_second': 497.294,
 'eval_steps_per_second': 31.192}

In [55]:
# Make a dataframe with the predictions and the text and the labels
import pandas as pd
import torch

items_for_manual_review = tokenized_dataset["test"].select(
    [0, 1, 22, 31, 43, 292, 448, 487]
)

baseline_predictions = []
index = 1
for i in items_for_manual_review:
    input_tokens = tokenizer(i['sms'], padding="max_length", truncation=True, return_tensors="pt")
    model = model.to('cuda')
    input_tokens = input_tokens.to('cuda')
    print(f"Processing item #{index}")
    index += 1
    with torch.no_grad():
        logits = model(**input_tokens).logits
        predicted_class_id = logits.argmax().item()
        baseline_predictions.append(predicted_class_id)

df = pd.DataFrame(
    {
        "sms": [item["sms"] for item in items_for_manual_review],
        "baseline_predictions": baseline_predictions,
        "labels": [item["label"] for item in items_for_manual_review],
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

Processing item #1
Processing item #2
Processing item #3
Processing item #4
Processing item #5
Processing item #6
Processing item #7
Processing item #8


,sms,baseline_predictions,labels
0,Yup... Hey then one day on fri we can ask miwa and jiayin take leave go karaoke \n,0,0
1,Happy new years melody!\n,0,0
2,PRIVATE! Your 2003 Account Statement for shows 800 un-redeemed S. I. M. points. Call 08715203652 Identifier Code: 42810 Expires 29/10/0\n,0,1
3,URGENT! We are trying to contact U. Todays draw shows that you have won a £800 prize GUARANTEED. Call 09050003091 from land line. Claim C52. Valid 12hrs only\n,0,1
4,I had askd u a question some hours before. Its answer\n,0,0
5,"SMS. ac JSco: Energy is high, but u may not know where 2channel it. 2day ur leadership skills r strong. Psychic? Reply ANS w/question. End? Reply END JSCO\n",0,1
6,"Yun ah.the ubi one say if ü wan call by tomorrow.call 67441233 look for irene.ere only got bus8,22,65,61,66,382. Ubi cres,ubi tech park.6ph for 1st 5wkg days.èn\n",0,0
7,Burger King - Wanna play footy at a top stadium? Get 2 Burger King before 1st Sept and go Large or Super with Coca-Cola and walk out a winner\n,0,1


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [56]:
from peft import (
    LoraConfig, get_peft_model, TaskType
)
from transformers import (
    AutoModelForSequenceClassification, AutoTokenizer
)
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# Make sure the model is aware of the tokenizer's pad token
config = LoraConfig(
    task_type=TaskType.SEQ_CLS, 
    inference_mode=False, 
    r=8, 
    target_modules=["q_lin", "v_lin"],
    lora_alpha=32, 
    lora_dropout=0.1
)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

# for name, module in model.named_modules():
#     print(name)
lora_model = get_peft_model(model, config)

# Inspect trainable params.
lora_model.print_trainable_parameters()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,331,716 || all params: 67,694,596 || trainable%: 1.967241225577297


In [57]:
def tokenize_and_add_labels(batch):
    tokenized = tokenizer(
        batch["sms"],
        truncation=True,
    )
    return tokenized
    
for split in splits:
    tokenized_dataset[split] = split_dataset[split].map(
        tokenize_and_add_labels, batched=True
    )
    
print(tokenized_dataset["train"][0])
print(tokenized_dataset["test"][0])
print(tokenized_dataset["train"][0].keys())

lora_trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam",
        # Set the learning rate
        learning_rate=2e-5,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",  
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

lora_trainer.train()

{'sms': 'I wanted to ask ü to wait 4 me to finish lect. Cos my lect finishes in an hour anyway.\n', 'label': 0, 'input_ids': [101, 1045, 2359, 2000, 3198, 1057, 2000, 3524, 1018, 2033, 2000, 3926, 3393, 6593, 1012, 2522, 2015, 2026, 3393, 6593, 12321, 1999, 2019, 3178, 4312, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'sms': 'Yup... Hey then one day on fri we can ask miwa and jiayin take leave go karaoke \n', 'label': 0, 'input_ids': [101, 9805, 2361, 1012, 1012, 1012, 4931, 2059, 2028, 2154, 2006, 10424, 2072, 2057, 2064, 3198, 2771, 4213, 1998, 25871, 25811, 2202, 2681, 2175, 13173, 11045, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
dict_keys(['sms', 'label', 'input_ids', 'attention_mask'])


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.074884,0.976703
2,0.161800,0.069799,0.976703


Checkpoint destination directory ./data/spam_not_spam/checkpoint-314 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/spam_not_spam/checkpoint-628 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=628, training_loss=0.1373261327196838, metrics={'train_runtime': 44.1196, 'train_samples_per_second': 227.382, 'train_steps_per_second': 14.234, 'total_flos': 168769528290048.0, 'train_loss': 0.1373261327196838, 'epoch': 2.0})

In [58]:
lora_trainer.evaluate()

{'eval_loss': 0.06979918479919434,
 'eval_accuracy': 0.9767025089605734,
 'eval_runtime': 1.1248,
 'eval_samples_per_second': 496.071,
 'eval_steps_per_second': 31.116,
 'epoch': 2.0}

In [59]:
# Make a dataframe with the predictions and the text and the labels
import pandas as pd

items_for_manual_review = tokenized_dataset["test"].select(
    [0, 1, 22, 31, 43, 292, 448, 487]
)

results = lora_trainer.predict(items_for_manual_review)
df = pd.DataFrame(
    {
        "sms": [item["sms"] for item in items_for_manual_review],
        "predictions": results.predictions.argmax(axis=1),
        "labels": results.label_ids,
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

,sms,predictions,labels
0,Yup... Hey then one day on fri we can ask miwa and jiayin take leave go karaoke \n,0,0
1,Happy new years melody!\n,0,0
2,PRIVATE! Your 2003 Account Statement for shows 800 un-redeemed S. I. M. points. Call 08715203652 Identifier Code: 42810 Expires 29/10/0\n,1,1
3,URGENT! We are trying to contact U. Todays draw shows that you have won a £800 prize GUARANTEED. Call 09050003091 from land line. Claim C52. Valid 12hrs only\n,1,1
4,I had askd u a question some hours before. Its answer\n,0,0
5,"SMS. ac JSco: Energy is high, but u may not know where 2channel it. 2day ur leadership skills r strong. Psychic? Reply ANS w/question. End? Reply END JSCO\n",0,1
6,"Yun ah.the ubi one say if ü wan call by tomorrow.call 67441233 look for irene.ere only got bus8,22,65,61,66,382. Ubi cres,ubi tech park.6ph for 1st 5wkg days.èn\n",1,0
7,Burger King - Wanna play footy at a top stadium? Get 2 Burger King before 1st Sept and go Large or Super with Coca-Cola and walk out a winner\n,0,1


###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [60]:
# Saving the model
lora_model.save_pretrained("./gpt-lora", save_adapter=True, save_config=True)
lora_model.merge_and_unload()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [61]:
from peft import (
    PeftConfig, AutoPeftModelForSequenceClassification
)
peft_model_id = "./gpt-lora"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoPeftModelForSequenceClassification.from_pretrained(peft_model_id)
model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(
                  in_features=768, out_features=768, bias=True
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=8, bias=Fals

In [62]:
# Make a dataframe with the predictions and the text and the labels
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

def tokenize_and_add_labels(batch):
    tokenized = tokenizer(
        batch["sms"],
        truncation=True,
    )
    return tokenized
    
for split in splits:
    tokenized_dataset[split] = split_dataset[split].map(
        tokenize_and_add_labels, batched=True,
    )

items_for_manual_review = tokenized_dataset["test"].select(
    [0, 1, 22, 31, 43, 292, 448, 487]
)

predictions = []
index = 0
for i in items_for_manual_review:
    input_tokens = tokenizer(i['sms'], padding="max_length", truncation=True, return_tensors="pt")
    print(f"Processing item #{index}")
    index += 1
    with torch.no_grad():
        logits = model(**input_tokens).logits
        predicted_class_id = logits.argmax().item()
        predictions.append(predicted_class_id)

Processing item #0
Processing item #1
Processing item #2
Processing item #3
Processing item #4
Processing item #5
Processing item #6
Processing item #7


In [63]:
df = pd.DataFrame(
    {
        "sms": [item["sms"] for item in items_for_manual_review],
        "fine_tuned_trainer_output": results.predictions.argmax(axis=1),
        "baseline_predictions": baseline_predictions,
        "peft_predictions": predictions,
        "labels": [item["label"] for item in items_for_manual_review],
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

,sms,fine_tuned_trainer_output,baseline_predictions,peft_predictions,labels
0,Yup... Hey then one day on fri we can ask miwa and jiayin take leave go karaoke \n,0,0,0,0
1,Happy new years melody!\n,0,0,0,0
2,PRIVATE! Your 2003 Account Statement for shows 800 un-redeemed S. I. M. points. Call 08715203652 Identifier Code: 42810 Expires 29/10/0\n,1,0,1,1
3,URGENT! We are trying to contact U. Todays draw shows that you have won a £800 prize GUARANTEED. Call 09050003091 from land line. Claim C52. Valid 12hrs only\n,1,0,1,1
4,I had askd u a question some hours before. Its answer\n,0,0,0,0
5,"SMS. ac JSco: Energy is high, but u may not know where 2channel it. 2day ur leadership skills r strong. Psychic? Reply ANS w/question. End? Reply END JSCO\n",0,0,0,1
6,"Yun ah.the ubi one say if ü wan call by tomorrow.call 67441233 look for irene.ere only got bus8,22,65,61,66,382. Ubi cres,ubi tech park.6ph for 1st 5wkg days.èn\n",1,0,1,0
7,Burger King - Wanna play footy at a top stadium? Get 2 Burger King before 1st Sept and go Large or Super with Coca-Cola and walk out a winner\n,0,0,0,1


In [65]:
! pip install sklearn

from sklearn.metrics import accuracy_score

# Compute accuracy
baseline_acc = accuracy_score([item["label"] for item in items_for_manual_review], baseline_predictions)
acc = accuracy_score([item["label"] for item in items_for_manual_review], predictions)
print(f"Baseline Accuracy is: {baseline_acc:.2f}")
print(f"PEFT Accuracy is: {acc:.2f}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to 

ModuleNotFoundError: No module named 'sklearn'